In [3]:
#encoding=utf-8
import jieba
import json
import os
import os.path
import math
import jieba.analyse

In [102]:
def gen_idf_of_docs(folder, idf_file_name, my_dict = ""):
    df_dict = {}
    body_theme_list = get_body_theme_list_from_folder(folder)
    body_list = [body_theme[0] for body_theme in body_theme_list]
    if "" != my_dict: jieba.load_userdict(my_dict)
    for body in body_list:
        seg_list = jieba.cut(body, cut_all=False)
        token_list = [token for token in seg_list]
        update_df_dict(df_dict, token_list)
    
    num_all_doc = len(body_list)
    if 0 == num_all_doc:
        return
    
    idf_dict = {}
    print len(df_dict)
    #print num_all_doc/(df_dict[token] + 1)
    idf_dict = {token: math.log(num_all_doc/(df_dict[token] + 1)) for token in df_dict if 0 != num_all_doc/(df_dict[token] + 1)}
    #for token in df_dict:
    #    print num_all_doc/(df_dict[token] + 1)
    #    idf_dict[token] = math.log(num_all_doc/(df_dict[token] + 1))
    write_dict_into_file(idf_dict, idf_file_name)
    
def get_body_theme_of_json_file(file_name):
    with open(file_name) as file_obj:
        data = json.load(file_obj)
        body = data['body']
        theme_list = data['related_theme'].strip().split(' ')
        return body, theme_list
    
def get_body_theme_list_from_folder(folder):
    file_name_list = [os.path.join(root, name) for root, dirs, files in os.walk(folder) for name in files]
    #print file_name_list
    body_theme_list = [get_body_theme_of_json_file(file_name) for file_name in file_name_list]
    
    
    return body_theme_list
    
def update_df_dict(df_dict, cur_file_token_list):
    for token in cur_file_token_list:
        if token not in df_dict: df_dict[token] = 0
        df_dict[token] += 1
        
def write_dict_into_file(dict_data, file_name):
    with open(file_name, 'w') as file_obj:
        data = [key.encode('utf-8') + '\t' + str(dict_data[key]) for key in dict_data]
        file_obj.write("\n".join(data))

[u'\u7535\u5b50\u5546\u52a1']
[u'\u673a\u573a']
[u'\u4fe1\u606f\u5316', u'\u519c\u4e1a\u4fe1\u606f\u5316', u'\u590d\u5408\u80a5']
[u'\u4e09\u661f', u'\u4e09\u661f\u624b\u673a']
[u'\u8de8\u5883\u7535\u5546', u'\u7269\u6d41', u'\u7535\u5b50\u5546\u52a1']
[u'\u4e24\u4f1a']
[u'\u4e91\u8ba1\u7b97']
[u'\u4e24\u4f1a']
[u'\u4e92\u8054\u7f51\u91d1\u878d', u'\u94f6\u884c']
[u'\u4e0a\u6d77\u56fd\u4f01']
[u'\u94f6\u884c']
[u'\u7269\u6d41', u'\u5316\u5986\u54c1', u'\u7535\u5b50\u5546\u52a1']
[u'\u94f6\u884c']
[u'IPO\u6ce8\u518c\u5236', u'\u4e24\u4f1a']
[u'\u7535\u5b50\u5546\u52a1']
[u'\u65c5\u6e38']
[u'\u4e00\u5e26\u4e00\u8def']
[u'\u94f6\u884c']
[u'\u4eac\u4e1c', u'\u7269\u6d41', u'\u7535\u5b50\u5546\u52a1']
[u'\u5149\u4f0f', u'\u7269\u8054\u7f51', u'\u50a8\u80fd\u6280\u672f']
[u'P2P', u'\u4e92\u8054\u7f51\u91d1\u878d', u'\u7f51\u7edc\u8d37\u6b3e']
[u'\u94f6\u884c']
[u'\u7535\u5b50\u5546\u52a1']
[u'\u6210\u54c1\u6cb9']
[u'\u7535\u5b50\u5546\u52a1']
[u'\u70bc\u7126\u7164\u53ca\u7126\u70ad']
[u'\u8d

In [125]:
#获得所有新闻的idf统计-保存到my_idf.txt文件中；打印词数
#ycj_news_path = "/home/xiangzi777/workspace/python/DM/theme_description/Data/crawl_data/theme_name/ycj_news"
ycj_news_path = "/home/xiangzi777/workspace/python/Scrapy/yuncaijing/ycj_news"
gen_idf_of_docs(ycj_news_path, "my_idf.txt", my_dict="my_dict_topic.txt")

273312


In [104]:
#获取文件的tf-idf字典，格式：{file_name: top_tf-idf_token_list(default:20)}
def get_file_tf_idf(doc_folder, idf_path, stop_word_path = "", topK = 5):
    
    theme_tf_idf_dict = {}
    
    if "" != idf_path: jieba.analyse.set_idf_path(idf_path=idf_path)
    if "" != stop_word_path: jieba.analyse.set_stop_words(stop_word_path)
    body_theme_list = get_body_theme_list_from_folder(doc_folder)
    #body_list = [body_theme[0] for body_theme in body_theme_list]
    #theme_list_list = [body_theme[1] for body_theme in body_theme_list]
    #theme_list_body_dict = zip(theme_list_list, body_list)
    
    for body_theme in body_theme_list:
        #print len(body_theme)
        body = body_theme[0]
        theme_list = body_theme[1]
        tf_idf_list = jieba.analyse.extract_tags(body, topK=topK, withWeight=True)
        for theme_name in theme_list:
            for tf_idf_pair in tf_idf_list:
                update_theme_tf_idf_dict(theme_tf_idf_dict, theme_name, tf_idf_pair)
                
    sorted_tf_idf_dict = {theme_name: sorted(theme_tf_idf_dict[theme_name].iteritems(), key = lambda d:d[1], reverse=True) for theme_name in theme_tf_idf_dict}
    return sorted_tf_idf_dict

def update_theme_tf_idf_dict(theme_tf_idf_dict, theme_name, tf_idf_pair):
    if theme_name not in theme_tf_idf_dict: theme_tf_idf_dict[theme_name] = {}
    token, weight = tf_idf_pair
    #print token, theme_name
    if token not in theme_tf_idf_dict[theme_name]: theme_tf_idf_dict[theme_name][token] = 0
    theme_tf_idf_dict[theme_name][token] += weight
    

In [ ]:
#获得主题的tf-idf排名靠前的token，每个主题关联的新闻取前topK（默认为5）个关键词；
theme_tf_idf_dict = get_file_tf_idf(ycj_news_path, "my_idf.txt", "my_stop_words.txt", topK = 5)

In [106]:
#从theme_tf_idf_list
def get_theme_tokens(theme, theme_tf_idf_list, topK = 20):
    if theme not in theme_tf_idf_list:
        return
    i = 0
    for token_weight in theme_tf_idf_list[theme]:
        print token_weight[0], token_weight[1]
        i += 1
        if i >= topK: return
        

In [124]:
get_theme_tokens(u'微信', theme_tf_idf_dict, topK = 50)

互联网 4.70105410247
投资 4.57031073526
基金 3.37146088306
城市 2.46665860976
以苗药 2.25122245184
鞍钢 2.10193714337
支付宝 1.83155175705
中成药 1.70782392899
主营 1.70782392899
每经 1.64341085705
信息 1.63566221
消费者 1.56606443464
微信支付 1.52678480896
朋友圈 1.49286687466
购物 1.45269123743
通过 1.42282418097
王军 1.37573737434
银行 1.31044021
进行 1.25872314458
官方 1.25053729027
移动支付 1.24889099146
平台 1.18446492394
全景网 1.14784140363
内容 1.13473160398
政务 1.12715150944
长虹 1.11422130979
游戏 1.10171876278
功能 1.09495533077
腾讯 1.06311711633
消息 1.01787651621
酒店 1.01639723709
保险 0.999415594294
空调 0.959871843299
幸福 0.905553307107
工行 0.896128633615
证照 0.864890832608
储油 0.862715180622
阿里 0.861504888938
控制 0.861245986196
代孕 0.860873203334
索尼 0.815585574704
去年同期 0.813394912516
会员 0.810492712061
消费 0.800603265867
回访 0.791706457146
证券 0.788394546673
发布 0.785594791794
郑素贞 0.756630854614
mjtzb2 0.75199598341
控烟 0.741131138994


In [100]:
import pickle
pickle.dump(theme_tf_idf_dict, open('theme_tf_idf_list_01.pickle', 'w'))